# Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) 是一项机器学习 (ML) 服务，可自动从扫描文档中提取文本、手写内容和数据。

它超越了简单的光学字符识别 (OCR)，能够识别、理解并从表单和表格中提取数据。如今，许多公司需要手动从扫描文档（如 PDF、图像、表格和表单）中提取数据，或者通过需要手动配置（且在表单更改时经常需要更新）的简单 OCR 软件来提取。为了克服这些耗时耗力的手动流程，`Textract` 利用 ML 读取和处理任何类型的文档，无需人工干预即可准确提取文本、手写内容、表格和其他数据。

此示例演示了如何在 LangChain 中将 `Amazon Textract` 用作 DocumentLoader。

`Textract` 支持 `PDF`、`TIFF`、`PNG` 和 `JPEG` 格式。

`Textract` 支持这些[文档大小、语言和字符](https://docs.aws.amazon.com/textract/latest/dg/limits-document.html)。

In [1]:
%pip install --upgrade --quiet  boto3 langchain-openai tiktoken python-dotenv

In [2]:
%pip install --upgrade --quiet  "amazon-textract-caller>=0.2.0"

## 示例 1

第一个示例使用本地文件，该文件内部将发送到 Amazon Textract 同步 API [DetectDocumentText](https://docs.aws.amazon.com/textract/latest/dg/API_DetectDocumentText.html)。

本地文件或类似 HTTP:// 的 URL 端点对 Textract 仅限于单页文档。
多页文档必须驻留在 S3 中。此示例文件是 jpeg 格式。

In [ ]:
from langchain_community.document_loaders import AmazonTextractPDFLoader

loader = AmazonTextractPDFLoader("example_data/alejandro_rosalez_sample-small.jpeg")
documents = loader.load()

文件输出

In [10]:
documents

[Document(page_content='Patient Information First Name: ALEJANDRO Last Name: ROSALEZ Date of Birth: 10/10/1982 Sex: M Marital Status: MARRIED Email Address: Address: 123 ANY STREET City: ANYTOWN State: CA Zip Code: 12345 Phone: 646-555-0111 Emergency Contact 1: First Name: CARLOS Last Name: SALAZAR Phone: 212-555-0150 Relationship to Patient: BROTHER Emergency Contact 2: First Name: JANE Last Name: DOE Phone: 650-555-0123 Relationship FRIEND to Patient: Did you feel fever or feverish lately? Yes No Are you having shortness of breath? Yes No Do you have a cough? Yes No Did you experience loss of taste or smell? Yes No Where you in contact with any confirmed COVID-19 positive patients? Yes No Did you travel in the past 14 days to any regions affected by COVID-19? Yes No Patient Information First Name: ALEJANDRO Last Name: ROSALEZ Date of Birth: 10/10/1982 Sex: M Marital Status: MARRIED Email Address: Address: 123 ANY STREET City: ANYTOWN State: CA Zip Code: 12345 Phone: 646-555-0111 Emer

## 示例 2
下一个示例从 HTTPS 端点加载文件。
它必须是单页的，因为 Amazon Textract 要求所有多页文档都存储在 S3 上。

In [7]:
from langchain_community.document_loaders import AmazonTextractPDFLoader

loader = AmazonTextractPDFLoader(
    "https://amazon-textract-public-content.s3.us-east-2.amazonaws.com/langchain/alejandro_rosalez_sample_1.jpg"
)
documents = loader.load()

In [11]:
documents

[Document(page_content='Patient Information First Name: ALEJANDRO Last Name: ROSALEZ Date of Birth: 10/10/1982 Sex: M Marital Status: MARRIED Email Address: Address: 123 ANY STREET City: ANYTOWN State: CA Zip Code: 12345 Phone: 646-555-0111 Emergency Contact 1: First Name: CARLOS Last Name: SALAZAR Phone: 212-555-0150 Relationship to Patient: BROTHER Emergency Contact 2: First Name: JANE Last Name: DOE Phone: 650-555-0123 Relationship FRIEND to Patient: Did you feel fever or feverish lately? Yes No Are you having shortness of breath? Yes No Do you have a cough? Yes No Did you experience loss of taste or smell? Yes No Where you in contact with any confirmed COVID-19 positive patients? Yes No Did you travel in the past 14 days to any regions affected by COVID-19? Yes No Patient Information First Name: ALEJANDRO Last Name: ROSALEZ Date of Birth: 10/10/1982 Sex: M Marital Status: MARRIED Email Address: Address: 123 ANY STREET City: ANYTOWN State: CA Zip Code: 12345 Phone: 646-555-0111 Emer

## 示例 3

处理多页文档需要该文档位于 S3 上。示例文档位于 us-east-2 存储桶中，并且 Textract 必须在同一区域被调用才能成功，因此我们设置了客户端的 `region_name` 并将其传递给加载器，以确保 Textract 从 us-east-2 调用。您也可以让您的 notebook 在 us-east-2 中运行，将 `AWS_DEFAULT_REGION` 设置为 us-east-2，或者在不同环境中运行时，传递一个具有该区域名称的 boto3 Textract 客户端，如下面的单元格所示。

In [12]:
import boto3

textract_client = boto3.client("textract", region_name="us-east-2")

file_path = "s3://amazon-textract-public-content/langchain/layout-parser-paper.pdf"
loader = AmazonTextractPDFLoader(file_path, client=textract_client)
documents = loader.load()

现在获取页数来验证响应（打印完整响应会很长……）。我们期望有 16 页。

In [13]:
len(documents)

16

## 示例 4

您可以选择向 `AmazonTextractPDFLoader` 传递一个名为 `linearization_config` 的附加参数，该参数将决定 Textract 运行后解析器如何线性化文本输出。

In [ ]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
from textractor.data.text_linearization_config import TextLinearizationConfig

loader = AmazonTextractPDFLoader(
    "s3://amazon-textract-public-content/langchain/layout-parser-paper.pdf",
    linearization_config=TextLinearizationConfig(
        hide_header_layout=True,
        hide_footer_layout=True,
        hide_figure_layout=True,
    ),
)
documents = loader.load()

## 在 LangChain 链中使用 AmazonTextractPDFLoader（例如 OpenAI）

AmazonTextractPDFLoader 的使用方式与其他加载器相同，可以在链中使用。
Textract 本身确实有一个 [Query 功能](https://docs.aws.amazon.com/textract/latest/dg/API_Query.html)，它提供了与本示例中 QA 链类似的功能，但也值得关注。

In [14]:
# You can store your OPENAI_API_KEY in a .env file as well
# import os
# from dotenv import load_dotenv

# load_dotenv()

In [15]:
# Or set the OpenAI key in the environment directly
import os

os.environ["OPENAI_API_KEY"] = "your-OpenAI-API-key"

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

chain = load_qa_chain(llm=OpenAI(), chain_type="map_reduce")
query = ["Who are the autors?"]

chain.run(input_documents=documents, question=query)

' The authors are Zejiang Shen, Ruochen Zhang, Melissa Dell, Benjamin Charles Germain Lee, Jacob Carlson, Weining Li, Gardner, M., Grus, J., Neumann, M., Tafjord, O., Dasigi, P., Liu, N., Peters, M., Schmitz, M., Zettlemoyer, L., Lukasz Garncarek, Powalski, R., Stanislawek, T., Topolski, B., Halama, P., Gralinski, F., Graves, A., Fernández, S., Gomez, F., Schmidhuber, J., Harley, A.W., Ufkes, A., Derpanis, K.G., He, K., Gkioxari, G., Dollár, P., Girshick, R., He, K., Zhang, X., Ren, S., Sun, J., Kay, A., Lamiroy, B., Lopresti, D., Mears, J., Jakeway, E., Ferriter, M., Adams, C., Yarasavage, N., Thomas, D., Zwaard, K., Li, M., Cui, L., Huang,'